In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import copy
from tqdm import tqdm
import os
import pickle

# Ensure the device is GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Extract and save features

### D1 to D10

In [2]:
class FeatureExtractor(nn.Module):
    def __init__(self, num_components=128):
        super(FeatureExtractor, self).__init__()
        # Use ResNet18 as backbone
        resnet = models.resnet152(pretrained=True)
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # Flatten to get 2048-dim feature vector
        return x

def extract_and_save_features(base_path, save_dir):
    """
    Extract features from all datasets and save them
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize feature extractor
    feature_extractor = FeatureExtractor().to(device)
    feature_extractor.eval()

    # Define transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Process training datasets (D1 to D20)
    for i in range(1, 11):
        print(f"Processing dataset D{i}...")

        # Load dataset
        data = torch.load(f"{base_path}/train_data/{i}_train_data.tar.pth")
        images = data['data']  # Shape: [N, 32, 32, 3]

        # Get labels if available (only for D1)
        labels = data.get('targets', None)

        # Process images in batches
        batch_size = 64
        features_list = []

        with torch.no_grad():
            for j in tqdm(range(0, len(images), batch_size)):
                batch_images = images[j:j + batch_size]
                # Convert to torch tensor and process
                batch_tensors = torch.stack([
                    transform(img) for img in batch_images
                ]).to(device)

                # Extract features
                batch_features = feature_extractor(batch_tensors)
                features_list.append(batch_features.cpu().numpy())

        # Combine all features
        features = np.concatenate(features_list, axis=0)

        # Save features and labels
        save_dict = {
            'features': features,
            'labels': labels
        }
        with open(f"{save_dir}/{i}_train_features.pkl", 'wb') as f:
            pickle.dump(save_dict, f)

        # Process corresponding test dataset
        print(f"Processing test dataset D{i}_test...")
        test_data = torch.load(f"{base_path}/eval_data/{i}_eval_data.tar.pth")
        test_images = test_data['data']
        test_labels = test_data['targets']

        test_features_list = []
        with torch.no_grad():
            for j in tqdm(range(0, len(test_images), batch_size)):
                batch_images = test_images[j:j + batch_size]
                batch_tensors = torch.stack([
                    transform(img) for img in batch_images
                ]).to(device)
                batch_features = feature_extractor(batch_tensors)
                test_features_list.append(batch_features.cpu().numpy())

        test_features = np.concatenate(test_features_list, axis=0)

        # Save test features and labels
        test_save_dict = {
            'features': test_features,
            'labels': test_labels
        }
        with open(f"{save_dir}/{i}_test_features.pkl", 'wb') as f:
            pickle.dump(test_save_dict, f)

In [3]:
if __name__ == "__main__":
    # Step 1: Extract and save features
    base_path = "../dataset/part_one_dataset"
    features_dir = "features"

    print("Phase 1: Extracting features...")
    extract_and_save_features(base_path, features_dir)

Phase 1: Extracting features...


/home/pankaj/anaconda3/envs/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pankaj/anaconda3/envs/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_39390/81333826.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob

Processing dataset D1...


100%|██████████| 40/40 [00:22<00:00,  1.77it/s]
/tmp/ipykernel_39390/81333826.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load(f"{base_path}/eval_

Processing test dataset D1_test...


...

100%|██████████| 40/40 [00:23<00:00,  1.73it/s]


Processing dataset D10...


100%|██████████| 40/40 [00:23<00:00,  1.73it/s]


Processing test dataset D10_test...


100%|██████████| 40/40 [00:23<00:00,  1.73it/s]


### D11 to D20

In [4]:
class FeatureExtractor(nn.Module):
    def __init__(self, num_components=128):
        super(FeatureExtractor, self).__init__()
        # Use ResNet18 as backbone
        resnet = models.resnet152(pretrained=True)
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # Flatten to get 2048-dim feature vector
        return x

def extract_and_save_features(base_path, save_dir):
    """
    Extract features from all datasets and save them
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Initialize feature extractor
    feature_extractor = FeatureExtractor().to(device)
    feature_extractor.eval()

    # Define transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Process training datasets (D1 to D20)
    for i in range(1, 11):
        print(f"Processing dataset D{i+10}...")

        # Load dataset
        data = torch.load(f"{base_path}/train_data/{i}_train_data.tar.pth")
        images = data['data']  # Shape: [N, 32, 32, 3]

        # Get labels if available (only for D1)
        labels = data.get('targets', None)

        # Process images in batches
        batch_size = 64
        features_list = []

        with torch.no_grad():
            for j in tqdm(range(0, len(images), batch_size)):
                batch_images = images[j:j + batch_size]
                # Convert to torch tensor and process
                batch_tensors = torch.stack([
                    transform(img) for img in batch_images
                ]).to(device)

                # Extract features
                batch_features = feature_extractor(batch_tensors)
                features_list.append(batch_features.cpu().numpy())

        # Combine all features
        features = np.concatenate(features_list, axis=0)

        # Save features and labels
        save_dict = {
            'features': features,
            'labels': labels
        }
        with open(f"{save_dir}/{i+10}_train_features.pkl", 'wb') as f:
            pickle.dump(save_dict, f)

        # Process corresponding test dataset
        print(f"Processing test dataset D{i+10}_test...")
        test_data = torch.load(f"{base_path}/eval_data/{i}_eval_data.tar.pth")
        test_images = test_data['data']
        test_labels = test_data['targets']

        test_features_list = []
        with torch.no_grad():
            for j in tqdm(range(0, len(test_images), batch_size)):
                batch_images = test_images[j:j + batch_size]
                batch_tensors = torch.stack([
                    transform(img) for img in batch_images
                ]).to(device)
                batch_features = feature_extractor(batch_tensors)
                test_features_list.append(batch_features.cpu().numpy())

        test_features = np.concatenate(test_features_list, axis=0)

        # Save test features and labels
        test_save_dict = {
            'features': test_features,
            'labels': test_labels
        }
        with open(f"{save_dir}/{i+10}_test_features.pkl", 'wb') as f:
            pickle.dump(test_save_dict, f)

In [5]:
if __name__ == "__main__":
    # Step 1: Extract and save features
    base_path = "../dataset/part_two_dataset"
    features_dir = "features"

    print("Phase 1: Extracting features...")
    extract_and_save_features(base_path, features_dir)

Phase 1: Extracting features...


/tmp/ipykernel_39390/3879576177.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(f"{base_path}/train_data/{i}_train_data.tar.pth")


Processing dataset D11...


100%|██████████| 40/40 [00:23<00:00,  1.73it/s]
/tmp/ipykernel_39390/3879576177.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load(f"{base_path}/eva

Processing test dataset D11_test...


...

100%|██████████| 40/40 [00:23<00:00,  1.73it/s]


Processing dataset D20...


100%|██████████| 40/40 [00:23<00:00,  1.73it/s]


Processing test dataset D20_test...


100%|██████████| 40/40 [00:23<00:00,  1.73it/s]
